# Tool Calling

Tool calling (also known as function calling) is a pattern where AI systems determine which tools or functions to execute based on user input. The most common modern approach uses large language models (LLMs) to analyze user requests and decide which functions to call.

## Semantic Tool Calling vs. LLM Tool Calling

**LLM-based tool calling** involves sending user queries to models like GPT-4 or Claude with function definitions, letting the LLM decide which tools to use:
- **Cost per request**: Each decision requires an API call
- **Latency**: Network round-trips add delay to every tool selection

**Semantic tool calling** uses vector embeddings and similarity matching to route queries to appropriate tools:
- **Speed**: Near-instantaneous tool selection through vector similarity
- **Cost-effective**: No API costs after initial setup

## How Semantic Tool Calling Works

1. **Define routes**: Create semantic routes for each tool with example user phrases
2. **Set thresholds**: Configure similarity requirements for route matching
3. **Route instantly**: Vector similarity determines the best tool match
4. **Execute**: Trigger the appropriate function based on semantic similarity

In this notebook, we'll implement semantic tool calling using RedisVL to route user queries to weather, FAQ, and notification tools - demonstrating a fast, cost-effective alternative to LLM-based tool selection.

## Installing dependencies
We'll need two main libraries for this semantic classification example:

RedisVL - A Python library that provides vector database capabilities and semantic routing functionality on top of Redis. RedisVL handles the vector storage, similarity search, and routing logic that powers our semantic classifier.

HuggingFace Sentence Transformers - A library that provides pre-trained models for converting text into high-quality sentence embeddings. These models have been specifically trained to capture semantic meaning, making them ideal for similarity-based classification tasks.

The sentence transformers library will automatically download the embedding model on first use. We're using a lightweight model (all-MiniLM-L6-v2) that provides good performance while being fast enough for real-time classification.

In [13]:
%pip install -q "redisvl>=0.8.2" sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Setting up the Text Vectorizer

The vectorizer is responsible for converting text into numerical vector representations that capture semantic meaning. RedisVL provides several vectorizer options such as OpenAI and VertexAI. We're using the HuggingFace Text Vectorizer for this example.

In [14]:
from redisvl.utils.vectorize import HFTextVectorizer

vectorizer = HFTextVectorizer()

11:55:09 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
11:55:09 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


## Defining Tool Routes

Each route represents a specific tool or function that users might want to access. We define routes by providing example phrases that represent different ways users might express the same intent.

In [15]:
from redisvl.extensions.router import Route, SemanticRouter

get_weather_route = Route(
    name="get_weather_route",
    references=[
        "What's the weather like?",
        "What's the weather today?",
        "Will it rain today?",
        "Is it raining?",
    ],
    distance_threshold=0.3
)

faq_route = Route(
    name="faq_route",
    references=[
        "What can you do?",
        "What are you capable of?",
        "What problems can you solve?",
        "How can you help me?",
    ],
    distance_threshold=0.3
)

notifications_route = Route(
    name="notifications_route",
    references=[
        "Do I have any notifications?",
        "Read my notifications",
        "Have I got notifications?",
    ],
    distance_threshold=0.3
)

## Creating the Semantic Router

The SemanticRouter is the central component that orchestrates the classification process. It combines your routes, vectorizer, and Redis storage to provide fast semantic classification capabilities.

In [16]:
tool_calling_router = SemanticRouter(
    name="tool_calling_router2",
    vectorizer=vectorizer,
    routes=[
        get_weather_route,
        faq_route,
        notifications_route,
    ],
    overwrite=True,
)

## Testing our semantic tool calling solution

In [17]:
user_query = "Do I have new notifications?"

route_match = tool_calling_router(user_query)

route_match

RouteMatch(name='notifications_route', distance=0.126499414444)

In [18]:
user_query = "Will it be sunny today?"

route_match = tool_calling_router(user_query)

route_match

RouteMatch(name='get_weather_route', distance=0.193976283073)

In [19]:
user_query = "What tasks are you capable of?"

route_match = tool_calling_router(user_query)

route_match

RouteMatch(name='faq_route', distance=0.214198470116)

In [20]:
user_query = "British Pubs are great!"

route_match = tool_calling_router(user_query)

route_match

RouteMatch(name=None, distance=None)

## Semantic Guardrails

In [21]:
from redisvl.extensions.router import Route, SemanticRouter
from redisvl.utils.vectorize import HFTextVectorizer

vectorizer = HFTextVectorizer()

block_route = Route(
    name="block_route",
    references=[
        "things about aliens",
        "corporate questions about agile",
        "anything about the S&P 500",
    ]
)

block_router = SemanticRouter(
    name="block_router",
    vectorizer=vectorizer,
    routes=[
        block_route
    ],
    overwrite=True,
)

11:57:19 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
11:57:19 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


In [22]:
user_query = "Is scrum a good framework?"

route_match = block_router(user_query)

route_match

RouteMatch(name='block_route', distance=0.429593980312)

In [23]:
user_query = "What are you capable of?"

route_match = block_router(user_query)

route_match

RouteMatch(name=None, distance=None)